# Opening a restaurant in Prague

In this project I look at the city of Prague in the Czech Republic to try to find the best neighborhood to open a restaurant.

In [6]:
# Importing the necessary libraries

import pandas as pd
from pandas import DataFrame
import numpy as np

import requests # library to handle requests
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# for the geolocation I use the OpenCage API
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
key = '9162917515ad475792f22cd5ca637f95'
geocoder = OpenCageGeocode(key)

# for the maps I use folium and some of its pluggins
import folium 
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

# to remove the accents
! pip install Unidecode
import unidecode 

# to calculate distances between coordinates
!pip install haversine
import haversine as hs

!pip install tabula-py

print("All packages loaded!")

All packages loaded!


# Prague's neighborhoods

First I retrieve a list of Prague's neighborhood from Wikipedia.

In [3]:
data_raw = pd.read_html('https://cs.wikipedia.org/wiki/Seznam_katastr%C3%A1ln%C3%ADch_%C3%BAzem%C3%AD_v_Praze', header = 0)[0]
data_raw.head()

,Pořadí,Katastrální území,Sčítání 2001,Evidence 2011,Evidence 2014[2],Rozloha (ha),hustota zalidnění (obyv/km²)
0,1,Stodůlky,52 101,59 711,61 105,962,6 351
1,2,Žižkov,55 401,55 691,56 829,544,10 443
2,3,Chodov,58 140,54 659,53 771,743,7 237
3,4,Vinohrady,54 516,50 720,50 751,379,13 401
4,5,Vršovice,36 345,37 066,35 930,293,12 243


I only need the name of the neighborhoods so I can drop the rest and rename the column. 

In [4]:
neighborhoods_cs = data_raw[['Katastrální území']]
neighborhoods_cs.rename(columns={'Katastrální území':'Neighborhoods_cs'}, inplace=True)
neighborhoods_cs.head()

/Users/fclauss/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Neighborhoods_cs
0,Stodůlky
1,Žižkov
2,Chodov
3,Vinohrady
4,Vršovice


Now I request the coordinates for each neighborhoods using the Opencage API and add them into the dataframe.
I use the opportunity to "standardize" the names of the neighborhoods by removing the accents.

In [8]:
# I first add the columns 
ls_lat = [] # this list will become the column 'latitude'
ls_lng = [] # this list will become the column 'longitude'
city_parts = [] # this list will become the column 'Neighborhoods' with the names without diacritics

# I define the function to retrieve the coordinate and add them into the respective lists
def geocoding(neigh):
    results = geocoder.geocode(neigh + ", Praha")
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']       
    ls_lat.append(lat)
    ls_lng.append(lng)
    city_parts.append(unidecode.unidecode(neigh))

# I call the function for all districts 
[geocoding(x) for x in neighborhoods_cs['Neighborhoods_cs']]

# finally I create the columns from the lists
neighborhoods_cs['Neighborhood'] = np.asarray(city_parts)
neighborhoods_cs['latitude'] = np.asarray(ls_lat)
neighborhoods_cs['longitude'] = np.asarray(ls_lng)

neighborhoods_cs.head()

/Users/fclauss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fclauss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fclauss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,Neighborhoods_cs,Neighborhood,latitude,longitude
0,Stodůlky,Stodulky,50.048307,14.312404
1,Žižkov,Zizkov,50.081054,14.454917
2,Chodov,Chodov,50.032843,14.501643
3,Vinohrady,Vinohrady,50.075359,14.436394
4,Vršovice,Vrsovice,50.071885,14.472665


This looks good and I can now remove the first column.

In [9]:
neighborhoods = neighborhoods_cs.drop(columns='Neighborhoods_cs')

Now I retrieve the coordinate for the city and use them to visualize the Prague and its districts.

In [10]:
results = geocoder.geocode('Prague')
lat_p = results[0]['geometry']['lat']
lng_p = results[0]['geometry']['lng']  
print('The geograpical coordinates of Prague are {}, {}.'.format(lat_p, lng_p))

The geograpical coordinates of Prague are 50.0874654, 14.4212535.


In [11]:
# I use Folium for the map visualization
map_prague = folium.Map(location=[lat_p, lng_p], zoom_start=11)
for lat, lng, neigh in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neigh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='cadetblue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_prague)  
    
map_prague

*There is 1 small anomalie: the neighborhood "Tocna" got somehow located about 30km south-west of the city. I could fix this with a more specific request (i.e. "Tocna, Praha 12"), but since I will focus on the city center and Tocna is located about 14km away, I will just ignore it.*

While there are certainly excellent opportunities in the outer parts of the city, we will focus here on the larger center by setting a maximum distance of 5km from the center.

For this I start by calculating the distance from each neighborhood to the center. For this purpose the Haversine distance calculation seems to be the prefered method and an excellent library exists for this purpose.

In [12]:
distances = []
origin = (lat_p, lng_p)  # latitude and longitude from Prague
for lat, lng in zip(neighborhoods['latitude'], neighborhoods['longitude']):
    point = (lat, lng)
    distances.append(hs.haversine(origin, point))
neighborhoods['Distance[km]'] = np.asarray(distances)

neighborhoods.head()

,Neighborhood,latitude,longitude,Distance[km]
0,Stodulky,50.048307,14.312404,8.906001
1,Zizkov,50.081054,14.454917,2.505453
2,Chodov,50.032843,14.501643,8.356001
3,Vinohrady,50.075359,14.436394,1.726055
4,Vrsovice,50.071885,14.472665,4.057035


Now I select the row within the 5km perimeter.

In [13]:
prague = neighborhoods[neighborhoods['Distance[km]'] < 5].reset_index(drop=True)
prague

,Neighborhood,latitude,longitude,Distance[km]
0,Zizkov,50.081054,14.454917,2.505453
1,Vinohrady,50.075359,14.436394,1.726055
2,Vrsovice,50.071885,14.472665,4.057035
3,Holesovice,50.100616,14.437384,1.860683
4,Smichov,50.074946,14.404844,1.819018
5,Liben,50.106103,14.476626,4.460459
6,Nusle,50.056530,14.442035,3.745966
7,Nove Mesto,50.075938,14.419946,1.285223
8,Brevnov,50.085209,14.371788,3.538113
9,Dejvice,50.102556,14.391797,2.688993


That leaves us with 21 neighborhoods. Now, let's see the map again. I can now remove the Distance column.

In [ ]:
prague.drop(columns='Distance[km]', inplace=True)

In [14]:
# I use Folium for the map visualization
map_prague = folium.Map(location=[lat_p, lng_p], zoom_start=13)
for lat, lng, neigh in zip(prague['latitude'], prague['longitude'], prague['Neighborhood']):
    label = '{}'.format(neigh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=30,
        popup=label,
        color='black',
        fill=True,
        fill_color='white',
        fill_opacity=0.3,
        parse_html=False).add_to(map_prague)  
    
map_prague

On this map it is easy to see that all the neighborhoods close to the center are represented and located properly.

# Prague's venues

Now I will analyze the venues of these neighborhoods using the Foursquare API. 
For this I start by setting up my credentials.

In [15]:
CLIENT_ID = '1PMTYLQUAGOCHDVMVGB3ENLJUADY1QRTYGZK1FTAMBSNEJ4N' # your Foursquare ID
CLIENT_SECRET = '3MU3CON3UCSAXM3SEA4WXNZS1ZF30YSWC1J3W5VCHQQK1AVD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

For the request, I decide to use a radius of 500m which is about the radius of the marker in the map above. This prevents from leaving out important parts especially of the old town. However, this means that I will certainly have to deal with some duplicates as it can be seen with the large overlapping areas.

In [16]:
# I define a function to create the get url
def built_url(neigh_lat,neigh_long,radius=500):
    LIMIT=100
    radius=500
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url

In [17]:
# I define the function to get the nearby venues of each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
prague_venues = getNearbyVenues(names=prague['Neighborhood'],
                                   latitudes=prague['latitude'],
                                   longitudes=prague['longitude']
                                  )

Zizkov
Vinohrady
Vrsovice
Holesovice
Smichov
Liben
Nusle
Nove Mesto
Brevnov
Dejvice
Bubenec
Kosire
Troja
Podoli
Karlin
Stare Mesto
Stresovice
Mala Strana
Vysehrad
Hradcany
Radlice
Josefov


In [19]:
print(prague_venues.shape)
prague_venues.head()

(82, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zizkov,50.081054,14.454917,Chilli & Lime,50.080432,14.454244,Asian Restaurant
1,Vinohrady,50.075359,14.436394,Náměstí Míru,50.075136,14.436554,Plaza
2,Vinohrady,50.075359,14.436394,Párek v rohlíku – Ladislav Červený,50.075042,14.437258,Hot Dog Joint
3,Vinohrady,50.075359,14.436394,The Craft: Food & Beers,50.076067,14.435890,Burger Joint
4,Vinohrady,50.075359,14.436394,Vanille,50.074865,14.436825,Ice Cream Shop


Since we are looking at a good place for a restaurant, let's put the existing ones on the map.

In [20]:
prague_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()

,Venue
Neighborhood,
Bubenec,7
Dejvice,2
Holesovice,4
Hradcany,8
Karlin,5
Kosire,3
Liben,2
Mala Strana,15
Nove Mesto,2


Let's see how many unique categories there are.

In [21]:
len(prague_venues['Venue Category'].unique())

53

A closer look at this list could give us some insight.

In [22]:
categories = DataFrame(prague_venues['Venue Category'].drop_duplicates(keep='first')).reset_index(drop=True)
print(categories.shape)
categories.head(10)

(53, 1)


,Venue Category
0,Asian Restaurant
1,Plaza
2,Hot Dog Joint
3,Burger Joint
4,Ice Cream Shop
5,Theater
6,Café
7,Farmers Market
8,Park
9,Dog Run


I will combine all restaurant into one category in order to better filter them afterwards.

In [23]:
general_category = []
for venue in prague_venues['Venue Category']:
    if 'Restaurant' in venue:
        general_category.append('Restaurant')  
    else:
        general_category.append(venue)

prague_venues['General Category'] = np.asarray(general_category)
prague_venues.head(20)    

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,General Category
0,Zizkov,50.081054,14.454917,Chilli & Lime,50.080432,14.454244,Asian Restaurant,Restaurant
1,Vinohrady,50.075359,14.436394,Náměstí Míru,50.075136,14.436554,Plaza,Plaza
2,Vinohrady,50.075359,14.436394,Párek v rohlíku – Ladislav Červený,50.075042,14.437258,Hot Dog Joint,Hot Dog Joint
3,Vinohrady,50.075359,14.436394,The Craft: Food & Beers,50.076067,14.435890,Burger Joint,Burger Joint
4,Vinohrady,50.075359,14.436394,Vanille,50.074865,14.436825,Ice Cream Shop,Ice Cream Shop
5,Vinohrady,50.075359,14.436394,Divadlo na Vinohradech,50.075997,14.437041,Theater,Theater
6,Holesovice,50.100616,14.437384,Kavárna Liberál,50.100590,14.436274,Café,Café
7,Holesovice,50.100616,14.437384,Farmářský trh Heřmaňák,50.100663,14.435984,Farmers Market,Farmers Market
8,Holesovice,50.100616,14.437384,Park na Vltavské,50.099947,14.437679,Park,Park
9,Holesovice,50.100616,14.437384,Venčící Plácek,50.100357,14.437272,Dog Run,Dog Run


Now we can add the restaurants on the map.

In [24]:
# I use Folium for the map visualization
map_prague = folium.Map(location=[lat_p, lng_p], zoom_start=13)
for lat, lng, venue, cat in zip(prague_venues['Venue Latitude'], prague_venues['Venue Longitude'],\
                           prague_venues['Venue Category'], prague_venues['General Category']):
    if cat == 'Restaurant':
        label = '{}'.format(venue)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='Blue',
            fill=True,
            fill_color='Orange',
            fill_opacity=0.7,
            parse_html=False).add_to(map_prague)  
    
map_prague

From the map it is obvious that the number of entries in Foursquare will not be sufficient to give us a promissing location for a restaurant. 

Fortunately the location of other restaurants is definitely not the only parameter that comes to play when choosing a location. 

There are generally to main type of customers for a restaurant: the "locals", that is people living in the neighborhood, and the tourists, of course. 

# Local customers

Local customer are absolutely essential for any good restaurant. They have certain expectation in terms of quality and pricing, but if met they will return regularly and form the customer base.

For this analysis, I will look at the population density for each neighborhood to give us an indication of the potential customer population. *In future analysis it would be reasonable to consider neighborhoods close by as potential customer population as well since the distances are not very large*.

The table we worked on at the begining actually included the population density for all neighborhoods, so I'll just add the coresponding column to the "prague" dataframe. However I will have to reformat it, removing the spaces to get integers.

In [25]:
# I extract the raw density data to my dataframe and change the data to string
prague['density_raw'] = data_raw['hustota zalidnění (obyv/km²)'].astype(str) 

# I remove the space separating the 1000'
density_corrected = []
for dens in prague['density_raw']:
    if len(dens) == 6:
        dens_new = dens[:2] + dens[-3:]
        density_corrected.append(dens_new)
    else:
        dens_new = dens[:1] + dens[-3:]
        density_corrected.append(dens_new)

# and add the result to the dataframe, changing the data to integer        
prague['density'] = np.asarray(density_corrected).astype(int)
prague.drop(columns='density_raw', inplace=True) # I remove the original column

prague

,Neighborhood,latitude,longitude,Distance[km],density
0,Zizkov,50.081054,14.454917,2.505453,6351
1,Vinohrady,50.075359,14.436394,1.726055,10443
2,Vrsovice,50.071885,14.472665,4.057035,7237
3,Holesovice,50.100616,14.437384,1.860683,13401
4,Smichov,50.074946,14.404844,1.819018,12243
5,Liben,50.106103,14.476626,4.460459,7655
6,Nusle,50.056530,14.442035,3.745966,5657
7,Nove Mesto,50.075938,14.419946,1.285223,6080
8,Brevnov,50.085209,14.371788,3.538113,4759
9,Dejvice,50.102556,14.391797,2.688993,4493


My first intention here was to use a choropleth map to display the population density, unfortunately I was not able to find a usable map with the neighborhoods.

So, I decided to use the standard Folium visualization and use the colors of the markers to display the population density. For this, I'll analyse quickly the density parameter to help me choose the thresholds. 

In [26]:
prague.density.describe()

count       22.000000
mean      7301.909091
std       3005.854318
min       3138.000000
25%       4751.500000
50%       6794.000000
75%       9510.750000
max      13401.000000
Name: density, dtype: float64

I decide to create 5 groups separated by the following thresholds: 4.000, 6.000, 8.000 and 10.000 hab/km2. 

In [27]:
map_prague_density = folium.Map(location=[lat_p, lng_p], zoom_start=12)

for neigh, lat, lng, dens in zip(prague['Neighborhood'], prague['latitude'], \
                                prague['longitude'], prague['density']):
    label = '{}, {}'.format(neigh, dens)
    label = folium.Popup(label, parse_html=True)
    if (dens < 4000):  
        folium.CircleMarker(
            [lat, lng],
            radius=15,
            popup=label,
            fill=True,
            fill_opacity=0.8,
            color=None,
            fill_color='#ffe6e6',
            parse_html=False).add_to(map_prague_density)
    elif (4000 < dens <= 6000):
        folium.CircleMarker(
            [lat, lng],
            radius=15,
            popup=label,
            fill=True,
            fill_opacity=0.8,
            color=None,
            fill_color='#ff8080',       
            parse_html=False).add_to(map_prague_density)  
    elif (6000 < dens <= 8000):
        folium.CircleMarker(
            [lat, lng],
            radius=15,
            popup=label,
            fill=True,
            fill_opacity=0.8,
            color=None,
            fill_color='#ff1a1a',       
            parse_html=False).add_to(map_prague_density) 
    elif (8000 < dens <= 10000):
        folium.CircleMarker(
            [lat, lng],
            radius=15,
            popup=label,
            fill=True,
            fill_opacity=0.8,
            color=None,
            fill_color='#990000',       
            parse_html=False).add_to(map_prague_density)
    elif dens > 10000:
        folium.CircleMarker(
            [lat, lng],
            radius=15,
            popup=label,
            fill=True,
            fill_opacity=0.8,
            color=None,
            fill_color='#4d0000',       
            parse_html=False).add_to(map_prague_density) 
              
map_prague_density

On the map we can see that the population is quite dense in the city center, but also in several neighborhoods outside. 
The neighborhoods Vinohrady, Smichov, Holesovice, Kosire and Stresovice all have a density over 10.000 habitants per square km.


# Tourists

Touristic flows through the streets of a city like Prague is very important. According to the statisitcs of Prague City Toursim (www.praguecitytourism.cz), in 2019 about 8 Million tourists visited the city.

The vast majority of tourist come to Prague for its impressive cultural heritage. This Wikipedia page (https://cs.wikipedia.org/wiki/Seznam_n%C3%A1rodn%C3%ADch_kulturn%C3%ADch_pam%C3%A1tek_%C4%8Cesk%C3%A9_republiky) provides a list of the main cultural monuments in the Czech Republic. We can use it to display the most touristic places on the city map.

In [28]:
monuments_raw = pd.read_html('https://cs.wikipedia.org/wiki/Seznam_n%C3%A1rodn%C3%ADch_kulturn%C3%ADch_pam%C3%A1tek_%C4%8Cesk%C3%A9_republiky', header = 0)[1]
data_raw.head()

,Pořadí,Katastrální území,Sčítání 2001,Evidence 2011,Evidence 2014[2],Rozloha (ha),hustota zalidnění (obyv/km²)
0,1,Stodůlky,52 101,59 711,61 105,962,6 351
1,2,Žižkov,55 401,55 691,56 829,544,10 443
2,3,Chodov,58 140,54 659,53 771,743,7 237
3,4,Vinohrady,54 516,50 720,50 751,379,13 401
4,5,Vršovice,36 345,37 066,35 930,293,12 243


I first filter the list for Prague monuments.

In [29]:
monuments_prague = monuments_raw[monuments_raw.Okres=='Praha']
list_monuments = monuments_prague[['Sídlo', 'Památka']]
list_monuments.rename(columns={'Sídlo':'Neighborhoods_raw', 'Památka': 'Monuments_raw'}, inplace=True)
list_monuments['Monument'] = list_monuments['Monuments_raw'].apply(lambda x: unidecode.unidecode(x))
list_monuments.head()

/Users/fclauss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Neighborhoods_raw,Monuments_raw,Monument
0,Praha 1 - Hradčany,Pražský hrad,Prazsky hrad
1,Praha 1 - Hradčany,české korunovační klenoty,ceske korunovacni klenoty
2,Praha 1 - Staré Město,kostel Panny Marie před Týnem,kostel Panny Marie pred Tynem
3,Praha 1 - Staré Město,palác Kinských,palac Kinskych
4,Praha 1 - Staré Město,Staroměstská radnice,Staromestska radnice


In [31]:
# I retrieve the latitude and longitude of the monuments

lattitudes = []
longitudes = []

for place in list_monuments['Monument']:
    try:
        results = geocoder.geocode(place)
        lati = results[0]['geometry']['lat']
        longi = results[0]['geometry']['lng']  
        lattitudes.append(lati)
        longitudes.append(longi)
    except:
        lati = 'NA'
        longi = 'NA'  
        lattitudes.append(lati)
        longitudes.append(longi)
list_monuments['Lattitude'] = np.asarray(lattitudes)
list_monuments['Longitude'] = np.asarray(longitudes)

list_monuments.head()

/Users/fclauss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fclauss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhoods_raw,Monuments_raw,Monument,Lattitude,Longitude
0,Praha 1 - Hradčany,Pražský hrad,Prazsky hrad,50.0908114,14.4005229
1,Praha 1 - Hradčany,české korunovační klenoty,ceske korunovacni klenoty,NA,NA
2,Praha 1 - Staré Město,kostel Panny Marie před Týnem,kostel Panny Marie pred Tynem,NA,NA
3,Praha 1 - Staré Město,palác Kinských,palac Kinskych,50.0880927,14.4216767
4,Praha 1 - Staré Město,Staroměstská radnice,Staromestska radnice,50.0869785,14.4202934


I can now display these touristic attractions onto the map

In [33]:
map_prague = folium.Map(location=[lat_p, lng_p], zoom_start=12)
for lat, lng, neigh in zip(list_monuments['Lattitude'], list_monuments['Longitude'], list_monuments['Monument']):
    if lat != 'NA':
        label = '{}'.format(neigh)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=8,
            popup=label,
            color='darkgrey',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_prague)
    else:
        pass
    
map_prague

The cultural monuments are unsurprisingly concentrated in the center of the city, more specifically in the neighborhoods: "Stare Mesto", "Josefov", "Vinohrady" and "Hradcany". So these are the neighborhood where most tourist will be located and will look for a place to eat.

# Conclusion

I analysed in this notebook Prague's neighborhoods to try to find a good place for a restaurant.

Using the data from FourSquare provided some entries, but by far not enough to help define a good location. 
I then turn to more data on the population density and touristic attractions with the rough assomption that the more people close by, the more potential customers.

The results show that, of course, most people and touristic attraction are concentrated in the very city center, namely Stare Mesto and Josefov, but the **Vinohrady neighborhood** could actually be a better place for a new restaurant as it combines the proximity with some major touristic attraction and a very high density of population.

Alternatively, one could decide to rather concentrate on the tourists, in which case Stare Mesto, Josefov or Hracany would be the best places. On the other side, avoiding the touristic places could be interesting to focus on the recurrent and more stable local customers. In this case, Holesovice, Stresovice and Kosire would be the prefered solutions. 

This analysis could be expended in many directions. For example, the rental costs could be an interesting information to add to the decision making process. Also the connections to the public transportation grid might be an important parameter.